In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os 

## 1.Get all links for zimuku items

In [2]:
## send the request
url = "http://www.zimuku.net/search?q=&t=onlyst"
result = requests.get(url)
print(result.status_code)

200


In [3]:
## get page number
c = result.content
soup = BeautifulSoup(c,"lxml")
pages =  soup.find('div',{'class':'pagination r clearfix'}).find('div')
page_num = pages.findAll('a')[-1].getText()
page_num = int(page_num)
print(page_num)


12993


In [4]:
## get download link from title link 
def get_down_link(url):
    #url = "http://www.zimuku.net/shooter/57265.html"
    try:
        result = requests.get(url)
        if result.status_code != 200:
            return None
        c = result.content
        soup = BeautifulSoup(c,"lxml")
        down_link =  soup.find('li',{'class':'li dlsub'}).find('a').get('href')
        return down_link
    except:
        return None

In [5]:
## download 
def download_zip(url,folder):
    try:
        result = requests.get(url)
        fname = result.headers['Content-Disposition'].split('"')[1]
        res= result.content
        with open( folder + fname, "wb" ) as f :
                f.write( res )
        return True
    except:
        return False

### set up some global variables

In [6]:
## create doc_list to hold all links 
doc_list = {'title':[],'link':[],'status':[]}
page_start = 9762
page_end = 10000                                ## process 3, 6303 to 10000 pages
folder = 'data/process3/data_download/'

# create folder if folder does not exist
if not os.path.exists(folder):
    os.makedirs(folder)        


In [ ]:

## loop through each page
for p in range(page_start,page_end):
    print(p-1, 'success')
    try:
        page_url = url + '&p=' + str(p)
        result = requests.get(page_url)
        #print(result.status_code)
        c = result.content
        soup = BeautifulSoup(c,"lxml")
        docs = soup.find('div',{'class':'box clearfix'}).findAll('div',{'class':'persub clearfix'})

        for doc in docs:

            ## get title 
            link_tag = doc.find('h1',{'class':'title'}).find('a')
            title = link_tag.getText().strip(' \t\n\r@').replace('\n','').replace('\r',' ')
            if title == '_':
                continue
            elif "/" in title:
                 title = title.split("/")[0]

            link='http://www.zimuku.net'+link_tag.get('href')

            ## get download link 
            down_link = get_down_link(link)

            doc_list['title'].append(title)
            doc_list['link'].append(down_link)
        
            ## download file 
            try:
                response = download_zip(down_link,folder)
                doc_list['status'].append(response)
            except:
                doc_list['status'].append(False)
    except:
        continue

print(len(doc_list['link']))

9761 success
9762 success
9763 success
9764 success
9765 success
9766 success
9767 success


In [ ]:
## export links file 
df = pd.DataFrame(doc_list)
df.to_csv('zimuku_links_v1.csv')
df.ix[0:5]

## Now start downloading